# Part 1 (50 points):  

Implement the Viterbi algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day 
is hot or cold based on the number of ice creams eaten).  See eisner.hmm.xls for a bit more discussion on the problem. 
Remember that the Viterbi algorithm computes the most likely sequence for an input. Your solution should be able to handle variable length sequences (in the range of 3-5).

In [183]:
import numpy as np

The following 3 files have observation/emission probabilities, transition probabilities and test sequence data (for 
evaluation), respectively:

In [184]:
testData = np.genfromtxt('http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv', delimiter=',')[1:,1:]
transitionProbs = np.genfromtxt('http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv', delimiter=',')
observationProbs = np.genfromtxt('http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv', delimiter=',')

The observation and transition probabilities have rows being the variable of interest, and columns being the 
conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label).  
The columns sum to 1.

The test data has one line per sequence.  When a sequence is less than five observations long, the last columns are 
filled with zeros.

In [185]:
class Viterbi:

    def __init__(self, transitionProbs, observationProbs):
        self.A = transitionProbs[1:-1, 1:-1]
        self.B = observationProbs[1:, 1:]
        self.P = transitionProbs[1:-1, -1:].T[0]
        #TODO use this for the last weight
        self.O = transitionProbs[-1:, 1:-1][0]

    def init_descent(self, testData):
        self.y = testData
        self.stateCount = self.A.shape[1]
        self.M = np.full((len(self.y), self.stateCount), float('nan'))
        self.C = np.full((len(self.y)-1, self.stateCount), float('nan'))
        for i in range(0, self.stateCount):
            self.M[0][i] = self.P[i] * self.B[int(self.y[i])-1][i]

    def row(self, x):
        maxidx = float('nan')
        for i in range(0, self.stateCount):
            candidates = np.full(self.stateCount, float('nan'))
            for j in range(0, self.stateCount):
                candidates[j] = self.A[j][i]*self.M[x-1][j]
            maxidx = np.where(candidates == np.nanmax(candidates))[0][0]
            self.M[x][i] = self.B[int(self.y[x])-1][i]*candidates[maxidx]
            if x < len(self.y):
                self.C[x-1][i] = maxidx + 1
            elif x > len(self.y):
                print("Invalid call")

    def descend(self):
        for i in range(1, len(self.y)):
            self.row(i)

    def ascend(self):
        predictions = np.full(len(self.y), float('nan'))
        bottom = len(predictions)-1
        predictions[-1] = np.where(self.M[bottom] == np.nanmax(self.M[bottom]))[0][0] + 1
        for i in range(bottom-1, -1, -1):
            predictions[i] = np.where(self.M[i+1] == np.nanmax(self.M[i+1]))[0][0] + 1
        return predictions

    def cleanPrint(self, results, messageWithItemNames):
        print(messageWithItemNames[0], end='')
        for result in results:
            print(f"{messageWithItemNames[int(result)]}  ", end='')
        print("\n")


In [194]:
vModel = Viterbi(transitionProbs,
                    observationProbs)

viterbi = []
for i, testCase in enumerate(testData):
    vModel.init_descent(np.trim_zeros(testCase))
    vModel.descend()
    viterbi.append(vModel.ascend())
    vModel.cleanPrint(viterbi[i], [f"Test {i+1}:  ", "Cold", "Hot"])

Test 1:  Hot  Hot  Hot  Hot  Hot  

Test 2:  Hot  Hot  Hot  Hot  

Test 3:  Cold  Hot  Hot  Cold  Cold  

Test 4:  Cold  Cold  Cold  

Test 5:  Cold  Cold  Hot  Hot  Hot  

Test 6:  Hot  Cold  Cold  Cold  

Test 7:  Hot  Hot  Hot  

Test 8:  Hot  Cold  Cold  Cold  

Test 9:  Hot  Hot  Hot  Hot  Hot  

Test 10:  Cold  Cold  Cold  



---

# Part 2 (50 points): 
Using the same network, implement likelihood (weighting) sampling for approximate inference.  For 
any test sequence, sample complete sequences of the hidden states n times, where n can range from 10 to 100000 samples. 
The goal is to approximate the likelihood of all possible sequences.

Assuming the Viterbi sequence is "correct", how long (how many samples) does it take the sampler to converge so that 
the (estimated) most likely sample you get matches the Viterbi sequence?

How do I sample a sequence? Short answer: The same as any other Bayesian Network.

Long answer: In essence, pick a length (3, 4, or 5) - pick the same lengths as each test sample.  Then, sample each 
weather-day (Hot/Cold) according to the distribution given by the transition network.  You will need to sample Day 1 
before sampling Day 2, for example.  You will then have a complete sample of sequence length 3/4/5).  The weight of that 
sequence sample will be the product of the observation probabilities given the sample (why?).  You can then judge by the 
overall weight which the most likely weather sequence would be.  Does the best string match your Viterbi answer?

Note: Technically, in the original problem there is the probability of sampling STOP given either HOT or COLD.  For this 
section of the homework, please just remove the STOP probability and renormalize the other two probabilities so that 
they sum to one.

In [195]:
x = (7/87)*0.06
TP = np.array([[0.87+(0.06-x), 0.5],
                   [0.07+x, 0.5]])
OP = observationProbs[1:,1:]
HOT = 1
rng = np.random.default_rng()

OP

array([[6.407e-01, 2.000e-04],
       [1.481e-01, 5.341e-01],
       [2.122e-01, 4.657e-01]])

In [386]:
def sampler(i1, i2, numSamples):
  x=0.0
  y=0.0
  samples = []
  i1=int(i1)-1
  i2=int(i2)-1
  for _ in range(numSamples):
    w1 = int(rng.random()<0.5)
    w2 = int(rng.random()<0.5)
    w = OP[i1][w1]*OP[i2][w2]
    if w1:
      x += w
    y+=w
  return x/y

In [391]:
for i, sequence in enumerate(testData):
  sequence = np.trim_zeros(sequence)
  guess = []
  for j in range(1, len(sequence)):
    w = sampler(sequence[j-1], sequence[j], 100)
    guess.append(w>0.5)
  print(guess)

  print(f'''
         Test {i} at {100} samples.
                                          
                            Viterbi: {viterbi[i]-1}
                           Sampling: {np.array(guess)}
        -----------------------------------------------
        ''')

[True, True, True, True]

         Test 0 at 100 samples.
                                          
                            Viterbi: [1. 1. 1. 1. 1.]
                           Sampling: [ True  True  True  True]
        -----------------------------------------------
        
[True, True, True]

         Test 1 at 100 samples.
                                          
                            Viterbi: [1. 1. 1. 1.]
                           Sampling: [ True  True  True]
        -----------------------------------------------
        
[True, False, True, True]

         Test 2 at 100 samples.
                                          
                            Viterbi: [0. 1. 1. 0. 0.]
                           Sampling: [ True False  True  True]
        -----------------------------------------------
        
[True, False]

         Test 3 at 100 samples.
                                          
                            Viterbi: [0. 0. 0.]
                           

In [380]:
def calcWeight(prYesterdayHot, numIcecreamSold, expected):
  x=0.0
  y=0.0
  n=0
  while (n < 10 or bool((x/y)>0.5)!=expected):
    yesterdayHot=int(rng.random()<prYesterdayHot)
    todayHot=int(rng.random()<OP[numIcecreamSold][HOT])
    weight=TP[yesterdayHot][todayHot]*OP[numIcecream][todayHot]
    if todayHot:
      weight+=TP[yesterdayHot][todayHot-1]*OP[numIcecream][todayHot-1]
      x+=weight
    else:
      weight+=TP[yesterdayHot][todayHot+1]*OP[numIcecream][todayHot+1]

    y+=weight

    n+=1

    if 10 < n < 20:
      print(f"weight:{weight:.3f}  {x:.3f}/{y:.3f}=={x/y:.3f}  calc|exp:{bool((x/y)>0.5)}|{expected}")

  return x/y, n

In [358]:
print(testData[2])

[3. 1. 3. 3. 1.]


In [379]:
for i in range(2,3):
  p = 0.5
  print(f'viterbi: {viterbi[i]}')
  for j in range(0, len(viterbi[i])):
    print(f'calcWeight({p}, {int(testData[i][j])-1}, {bool(viterbi[i][j]-1.)})')
    p, n = calcWeight(p, int(testData[i][j])-1, bool(viterbi[i][j]-1.))
    print("predicted {}")
  print("next")

viterbi: [1. 2. 2. 1. 1.]
calcWeight(0.5, 2, False)
weight:0.429  1.673/3.277==0.510  calc|exp:True|False
weight:0.249  1.922/3.526==0.545  calc|exp:True|False
weight:0.249  1.922/3.775==0.509  calc|exp:True|False
weight:0.429  1.922/4.204==0.457  calc|exp:False|False
predicted {}
calcWeight(0.457077916423574, 0, True)
weight:0.249  0.000/3.277==0.000  calc|exp:False|True
weight:0.249  0.000/3.526==0.000  calc|exp:False|True
weight:0.429  0.000/3.955==0.000  calc|exp:False|True
weight:0.429  0.000/4.384==0.000  calc|exp:False|True
weight:0.249  0.000/4.633==0.000  calc|exp:False|True
weight:0.249  0.000/4.882==0.000  calc|exp:False|True
weight:0.429  0.000/5.311==0.000  calc|exp:False|True
weight:0.249  0.000/5.560==0.000  calc|exp:False|True
weight:0.429  0.000/5.989==0.000  calc|exp:False|True


KeyboardInterrupt: ignored